In [154]:
from pymongo import MongoClient
import pandas as pd
import plotly.graph_objects as go

In [162]:
uri = "mongodb://127.0.0.1"
mongo = MongoClient(uri)

DC = {"lat": 38.93, "lon": -76.98}
LA = { "lat": 34.0522, "lon": -118.2437}

CENTER = LA

In [170]:
def get_path(route):
    lat, lng = [], []
    try:
        for point in route["data"]["data"]["directions"]["points"]:
            lat.append(point["lat"])
            lng.append(point["lng"])
        return lat, lng
    except:
        return None

db = mongo.main
fig = go.Figure()
fig.add_trace(go.Scattergeo(locationmode = 'USA-states'))

for ride in db.routes.find():
    pickup, destination = ride["pickup"], ride["destination"]
    if ride.get("prices") is None:
        continue
    prices = {x[0]: x[1] for x in ride["prices"]}
    if prices.get("UberX") is None:
        continue
    dollars_per_mile = float(prices["UberX"])/(ride["distance"]/1609.344)
    route = db.directions.find_one({"pickup": pickup, "destination": destination})
    if route is None:
        continue
    path = get_path(route)
    if path is None:
        continue
    fig.add_trace(go.Scattergeo(
        lat = path[0],
        lon = path[1],
        mode = "lines",
        line = dict(width = 0.5, color = f'rgb(0, {255-dollars_per_mile*40}, {dollars_per_mile*40})'),
        text = "${:.2f}/mi, {:.2f}mi".format(dollars_per_mile, ride["distance"]/1609.344),
        opacity = 0.2
        )
    )
    
fig.update_layout(
    geo = go.layout.Geo(
        center = CENTER,
        projection = { "scale": 500 }
    )
)
fig.show()